In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.show(1)

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0e72087-3bcf-4317-b047-84ce6ef5e58e;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (123ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (65ms)
:: resolution report :: resolve 4425ms :: artifacts dl 193ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------

+--------------------+---+----+---+-----------+
|             user_id|  x|   y|  t|pixel_color|
+--------------------+---+----+---+-----------+
|vKtmwRQwAYtw5poO4...|804|1992|  0|    #FF3881|
+--------------------+---+----+---+-----------+
only showing top 1 row



<div style="color:pink">

Idee: 
-Daten nach Nutzern aggregieren und die Zeit zwischen aufeinanderfolgenden Pixeln vergleichen[erledigt] <br>
-Zeit (Differenz) festhalten und mit dem nächsten Zeitraum vergleichen [erledigt] <br>
-Falls die Differenz abweicht, ist der Nutzer wahrscheinlich kein Bot(?) <br>

</div>

In [2]:
from src.features.feature_functions import group_dataframe_by_user
groupedFrame = group_dataframe_by_user(dataFrame)
#groupedFrame.show(5)
#brauchen wir nicht mehr :)

In [33]:
import pyspark.sql.functions as F
#Nutzung von dataFrame oder groupedFrame sinnvoller?
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t', 'df2.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
timeDiffFrame.show(15)

+--------------------+----+----+----+
|             user_id|  t1|  t2|diff|
+--------------------+----+----+----+
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
+--------------------+----+----+----+
only showing top 15 rows



<div style ="color:pink">
-Differenz speichern und beim nächsten Vergleich erneut verwenden? [erledigt] <br>
-Nutzer, die das Kriterium bei einem der Vergleiche nicht erfüllen, müssten nicht erneut betrachtet werden(?) <br>
</div>

In [35]:
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['df1.user_id', 'df1.t'])
timeDiffReduced.show(15)

22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:20:45 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+----+----+----+
|             user_id|  t1|  t2|diff|
+--------------------+----+----+----+
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 548| 870| 322|
|++/iKk3F3ZlK4sJxa...| 870|1201| 331|
|++/iKk3F3ZlK4sJxa...|1201|1553| 352|
|++/iKk3F3ZlK4sJxa...|1553|2174| 621|
|++/iKk3F3ZlK4sJxa...|2174|2487| 313|
|++/xB4v5Jeeacv4KO...|2037|2606| 569|
|++0X370V+dVFAmRQD...|2357|2660| 303|
|++0X370V+dVFAmRQD...|2660|2974| 314|
|++0X370V+dVFAmRQD...|2974|3291| 317|
|++0nCmngtZ68vbg6p...| 734|2690|1956|
|++0rQ+/WspcGyNaOH...|3008|3316| 308|
|++1HxWRIxcZdBaKrR...|1149|3310|2161|
+--------------------+----+----+----+
only showing top 15 rows



In [41]:
#timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'), 'inner')
botDiff = botDiff.where('tdr1.t1 <> tdr2.t1').where('tdr1.diff == tdr2.diff')
botDiff = botDiff.orderBy(['tdr1.user_id'])
botDiff = botDiff.select('tdr1.user_id', 'tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff.show(15)

22/06/06 17:31:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:31:37 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+----+
|             user_id|diff|
+--------------------+----+
|++5ngkIIPwfxJtzv3...| 307|
|++6IlQ1s33JtGR24y...| 316|
|++ItjDxLoRLVe7/C4...| 315|
|++Lvwnim+QaifD9U4...| 307|
|++UiyqDibdpjwLi8O...| 307|
|++ZTAhuLp4CF8NxY2...| 329|
|++quEyKIDZltXAJO5...| 311|
|++t/2ZoXH2RQwnaVK...| 315|
|+/6XJYftUnMwTgGPl...| 316|
|+/8RPTGDQ4veBkUkY...| 311|
|+02eF0+7cl3Ol1b64...| 304|
|+05i4u0UwrZ9JkLMX...| 309|
|+0AL86CS7hWfXsH5s...| 312|
|+0PwSvdOri/m9qWYQ...| 307|
|+0RlYPM6KSjW8ShiI...| 303|
+--------------------+----+
only showing top 15 rows



In [42]:
print(botDiff.count())

22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 17:32:41 WARN RowBasedKeyValueBatch: Calling spill() on

28205
